
# 📊 EDA Inicial — Abandono Laboral
**Autora:** Paola Navarro · **Equipo:** Mateo Beltramone, Gonzalo Ezequiel Luna  
**Objetivo:** Generar visualizaciones provisionales y redactar hallazgos iniciales.

> **Notas:**
> - Este cuaderno usa **solo matplotlib** para gráficos, **una figura por gráfico** y **sin estilos ni colores específicos**.
> - Ajustá los **nombres de columnas** según tu dataset.  
> - Las celdas marcadas como **(Opcional)** pueden omitirse si Gonzalo ya aplicó esa transformación en el dataset limpio.


In [ ]:

# ====== Librerías ======
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mostrar más columnas al inspeccionar
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


## 1) Cargar datos

In [ ]:

# Ruta al archivo (ajustar nombre/ubicación)
DATA_PATH = "abandono_laboral_limpio.csv"  # Ej.: generado por Gonzalo

# Cargar dataset
df = pd.read_csv(DATA_PATH)

# Vista rápida
display(df.head(3))
print("\nForma del dataset:", df.shape)
print("\nTipos de datos:")
display(df.dtypes)



## 2) (Opcional) Normalización de categorías  
Usar solo si **aún no** se unificaron `sexo` y `estado_civil`. Si ya están OK, **saltá esta celda**.


In [ ]:

# --- Unificar 'sexo' --- (editar mapeos si es necesario)
if "sexo" in df.columns:
    map_sexo = {
        "M": "Masculino", "F": "Femenino",
        "m": "Masculino", "f": "Femenino",
        "male": "Masculino", "female": "Femenino",
        "Masculino": "Masculino", "Femenino": "Femenino"
    }
    df["sexo"] = df["sexo"].astype(str).str.strip().str.lower().map(map_sexo).fillna(df["sexo"])

# --- Unificar 'estado_civil' --- (editar mapeos si es necesario)
if "estado_civil" in df.columns:
    df["estado_civil"] = (
        df["estado_civil"]
        .astype(str).str.strip().str.lower()
        .replace({
            "casado": "Casado/a", "casada": "Casado/a",
            "soltero": "Soltero/a", "soltera": "Soltero/a",
            "divorciado": "Divorciado/a", "divorciada": "Divorciado/a",
            "viudo": "Viudo/a", "viuda": "Viudo/a",
        })
    )

# Estandarizar target 'abandono' a 0/1 si aplica
if "abandono" in df.columns:
    # Intento de coerción suave
    df["abandono"] = df["abandono"].replace({"no":0, "sí":1, "si":1, "Yes":1, "No":0, True:1, False:0}).astype(int, errors="ignore")

display(df.head(3))


## 3) Selección de columnas útiles (ajustar según su dataset)

In [ ]:

# Sugerencia mínima de columnas (renombrar según corresponda)
# num_cols y cat_cols se pueden ajustar a mano si es necesario.
num_cols = [c for c in df.columns if df[c].dtype.kind in "if" and c not in ["abandono"]]
cat_cols = [c for c in df.columns if df[c].dtype == "object"]
target_col = "abandono" if "abandono" in df.columns else None

print("Numéricas:", num_cols)
print("Categóricas:", cat_cols)
print("Target:", target_col)


## 4) Estadísticas descriptivas

In [ ]:

# Estadísticas para numéricas
display(df[num_cols].describe().T)

# Frecuencias básicas para categóricas (máximo 20 categorías por columna para vista rápida)
for c in cat_cols:
    vc = df[c].value_counts(dropna=False)
    print(f"\nFrecuencias — {c}:")
    display(vc.head(20))


## 5) Histogramas — Variables numéricas

In [ ]:

for col in num_cols:
    fig = plt.figure(figsize=(6,4))
    plt.hist(df[col].dropna(), bins=20)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()


## 6) Barras — Variables categóricas

In [ ]:

for col in cat_cols:
    counts = df[col].value_counts(dropna=False)
    fig = plt.figure(figsize=(6,4))
    plt.bar(counts.index.astype(str), counts.values)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Cantidad")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


## 7) Boxplot — Salario vs Abandono (si aplica)

In [ ]:

# Editar el nombre de la columna de salario si es distinto
salario_col = "salario" if "salario" in df.columns else None

if target_col and salario_col and target_col in df.columns and salario_col in df.columns:
    # Preparar datos
    grupos = []
    etiquetas = []
    for val in sorted(df[target_col].dropna().unique()):
        grupos.append(df.loc[df[target_col] == val, salario_col].dropna().values)
        etiquetas.append(str(val))

    if len(grupos) >= 2:
        fig = plt.figure(figsize=(6,4))
        plt.boxplot(grupos, labels=etiquetas, showmeans=True)
        plt.title(f"{salario_col} vs {target_col}")
        plt.xlabel(f"{target_col} (1=Sí, 0=No)")
        plt.ylabel(salario_col)
        plt.show()


## 8) Tasa de abandono por categoría (porcentaje)

In [ ]:

def plot_abandono_rate_by(col):
    if target_col is None or col not in df.columns:
        return
    tmp = df[[col, target_col]].dropna()
    # Calcular tasa (%)
    rates = tmp.groupby(col)[target_col].mean().sort_values()
    fig = plt.figure(figsize=(6,4))
    plt.bar(rates.index.astype(str), (rates.values * 100.0))
    plt.title(f"Tasa de abandono por {col} (%)")
    plt.xlabel(col)
    plt.ylabel("Porcentaje de abandono")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

# Elegir algunas columnas categóricas clave si existen
for col in ["sexo", "estado_civil"]:
    if col in df.columns:
        plot_abandono_rate_by(col)


## 9) Matriz de correlación (numéricas)

In [ ]:

if len(num_cols) >= 2:
    corr = df[num_cols].corr(numeric_only=True)
    fig = plt.figure(figsize=(6,5))
    plt.imshow(corr, interpolation='nearest')
    plt.title("Correlación entre variables numéricas")
    plt.colorbar()
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
    plt.yticks(range(len(corr.index)), corr.index)
    plt.tight_layout()
    plt.show()
    display(corr.round(3))



## 10) Hallazgos preliminares (para redactar)
Completar estos puntos con observaciones surgidas de los gráficos/estadísticas:

- **Distribución de edad/salario/antigüedad:** _ej.: la mayor densidad se ubica entre X–Y; hay outliers en ..._
- **Categóricas clave (sexo, estado civil, departamento si aplica):** _ej.: desequilibrio de clases, categorías dominantes_
- **Abandono por grupos:** _ej.: la tasa de abandono es mayor en ..._
- **Relación salario vs abandono:** _ej.: mediana inferior en el grupo que abandona_
- **Correlaciones numéricas relevantes:** _ej.: antigüedad y salario correlacionan en +0.45_
- **Datos faltantes/outliers:** _ej.: % de nulos en ...; tratamiento sugerido_



## 11) Checklist ético (para revisión de Mateo)
- [ ] Variables sensibles (sexo, edad, estado civil) se usan **solo** para análisis legítimos.
- [ ] Se reportan **tasas** y no solo conteos para evitar sesgos por desbalance.
- [ ] Se evita inferir causalidad a partir de correlaciones/descriptivos.
- [ ] Se documentan **limitaciones** del dataset (cobertura, calidad, sesgos de muestreo).
